

For this practice we want to store the Yelp shops and review in MongoDB and use that to run some queries.
Before starting this notebook, create a yelp dataset on MongoDB, then add reviews and shops collection to it. The notes on Setting up MongoDB show how you can do this.

** Submission Instruction **

* Make a copy and replace blank with yourname
* Complete the notebook and run all the cells
* Download the .ipybn
* Submit on Gradescope

In [ ]:
!pip install yelpapi
!pip install "pymongo[srv]"

In [ ]:
# Libraries
from yelpapi import YelpAPI
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


In [ ]:

yelp_api = YelpAPI('ENTER YOUR KEY HERE')

In [ ]:
#connecting to MongoDB

#change uri with your connection string
uri = "mongodb+srv://<username>:<password>@data-eng.9tz1zzp.mongodb.net/?retryWrites=true&w=majority&appName=DATA-ENG"
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


In [ ]:
#making sure that shops and review collections have been created!

db = client['yelp']
shops_collection = db['shops']
reviews_collection = db['reviews']

Let's search for taco shops and store to taco_shops, this time in Illinois


In [ ]:
taco_shops = yelp_api.search_query(term = 'taco',  location = 'IL')

Let's first check if we got the shops correctly:

In [ ]:
taco_shops['businesses'][0]

Now we insert all of the documents to the shops collections:

`insert_many` is similar to `insert_one` but accept a list of document to insert.

In [ ]:
result = shops_collection.insert_many(taco_shops['businesses'])
result

Q1 [3 point]: Using find, list id (yelp id ) and name of all shops in the collection that are in Chicago. You need to use location.city

In [ ]:
chicago_shops = ...
for shop in chicago_shops:
  print(shop)

Note that in the above documents 'id' is the yelp id for the document and '_id' is the MonogoDB id for the document.

In [ ]:
#check  one review for one shop to see the structure:
reviews = yelp_api.reviews_query(id = 'htKi45T3U08n8FoHgTmLwg');
print(reviews['reviews'][0])

Now for each document in the shop query, we use yelp to retrieve the reviews  and insert them to reviews collection at MongoDB. Reviews do not have shop name and the title, so add them to the reviews.

In [ ]:
inserted_review = 0;
chicago_shops = shops_collection.find({}, {"id":1, "name":1})
for shop in chicago_shops:
  shop_id = shop['id'];
  shop_name = shop['name'];
  reviews = yelp_api.reviews_query(id = shop_id)
  for review in reviews['reviews']:
    review['shop_id'] = shop_id;
    review['shop_name'] = shop_name;
    reviews_collection.insert_one(review);
    inserted_review += 1;

print("Inserted ", inserted_review , " reviews into the reviews collection");

Q2: [4 points] Find all the reviews with the rating of greater than or equal to 4.5. Only keep the name of the shop, text, and rating (hint: use projection).

In [ ]:
results = reviews_collection....
for r in results:
  print(r)

Q3 [4 points]: using `time_created` field in the reviews, find of all reviews that have been entered in 2024 (after '2024-01-01)'. Only keep the shop_name, text, rating, and time_created.

In [ ]:
results = reviews_collection....
for r in results:
  print(r)

Ok, now let's find the average rating for each shop

In [ ]:
#To keep the the shop name, we can use "$first" which keep the first value of the requested field in all of the documents in the group.
#Since all of the documents would have the same shop name (we group them on shop_id), shop_name would be the same and $first gets the correct shop name

pipeline = [
    {
        "$group": {
            "_id": "$shop_id",
            "averageRating": { "$avg": "$rating" },
            "shop_name": { "$first": "$shop_name" }
        }
    },
    {
        "$project": {
            "shop_id": "$_id",
            "shop_name": 1,
            "averageRating": 1
        }
    }

]

result = reviews_collection.aggregate(pipeline)

# Print results
for doc in result:
    print(doc)

Q4 [4 points]: Find the averate rating of each shop for the only reviews that have been entered in 2024, and return top 10 shops with highest average rating (all with one pipeline)

In [ ]:
#Hint: you have to add a couple of stages to the previous pipline

pipeline = [

    {
        "$group": {
            "_id": "$shop_id",
            "averageRating": { "$avg": "$rating" },
            "shop_name": { "$first": "$shop_name" }
        }
    },
    {
        "$project": {
            "shop_id": "$_id",
            "shop_name": 1,
            "averageRating": 1
        }
    },

]

result = reviews_collection.aggregate(pipeline)

# Print results
for doc in result:
    print(doc)


In [ ]:
client.close()